# Answers to exercises

In [ ]:
# Import the usual stuff first
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
%matplotlib inline

# We'll need this too
import logomaker
import os.path 
from scipy.signal import convolve

This course will include a variety of exercises to increase your Python skills. Note that the knowledge needed to complete each exercise will NOT necessarily have been presented or discussed. If you find yourself at sea, **the first thing you should do is Google your question.** 

## 2_datatypes.ipynb

Here is the DNA sequence of the multiple cloning site (MCS) on the plasmid [pcDNA5](https://www.addgene.org/vector-database/2132/), a popular vector for mammalian gene expression.

In [ ]:
# Note how to define a long string over multiple lines
mcs_seq = 'GAGACCCAAGCTGGCTAGCGTTTAAACTTAAGCTTGGTACCGAGCTCGGATCCACTA' \
          'GTCCAGTGTGGTGGAATTCTGCAGATATCCAGCACAGTGGCGGCCGCTCGAGTCTAG' \
          'AGGGCCCGTTTAAACCCGCTGATCAGCCT'
print(mcs_seq)

**E2.1**: Does this MCS contain a restriction site for NheI (GCTAGC)? How about for MscI (TGGCCA)? 

In [ ]:
# Answer

site_NheI = 'GCTAGC'
site_MscI = 'TGGCCA'

print('NheI: ', site_NheI in mcs_seq)
print('MscI: ', site_MscI in mcs_seq)

**E2.2**: Using the string method `.find()`, find the location(s) of the above restriction sites within the MCS.

In [ ]:
# find site
site_start = mcs_seq.find(site_NheI)
print(f'site starts at position {site_start}')

# check
site_stop = site_start + len(site_NheI)
print('found site: ', mcs_seq[site_start:site_stop])
print('NheI site : ', site_NheI)

**E2.3**: Using the string method `.replace()`, compute the RNA sequence transcribed from the GFP gene sequence (given below). 

In [ ]:
gfp_seq = 'ATGAGTAAAGGAGAAGAACTTTTCACTGGAGTTGTCCCAATTCTTGTTGAATTAGATGGTGATG' \
          'TTAATGGGCACAAATTTTCTGTCAGTGGAGAGGGTGAAGGTGATGCAACATACGGAAAACTTAC' \
          'CCTTAAATTTATTTGCACTACTGGAAAACTACCTGTTCCATGGCCAACACTTGTCACTACTTTC' \
          'GCGTATGGTCTTCAATGCTTTGCGAGATACCCAGATCATATGAAACAGCATGACTTTTTCAAGA' \
          'GTGCCATGCCCGAAGGTTATGTACAGGAAAGAACTATATTTTTCAAAGATGACGGGAACTACAA' \
          'GACACGTGCTGAAGTCAAGTTTGAAGGTGATACCCTTGTTAATAGAATCGAGTTAAAAGGTATT' \
          'GATTTTAAAGAAGATGGAAACATTCTTGGACACAAATTGGAATACAACTATAACTCACACAATG' \
          'TATACATCATGGCAGACAAACAAAAGAATGGAATCAAAGTTAACTTCAAAATTAGACACAACAT' \
          'TGAAGATGGAAGCGTTCAACTAGCAGACCATTATCAACAAAATACTCCAATTGGCGATGGCCCT' \
          'GTCCTTTTACCAGACAACCATTACCTGTCCACACAATCTGCCCTTTCGAAAGATCCCAACGAAA' \
          'AGAGAGACCACATGGTCCTTCTTGAGTTTGTAACAGCTGCTGGGATTACACATGGCATGGATGA' \
          'ACTATACAAATAA'

# Answer here
gfp_rna = gfp_seq.replace('T','U')
gfp_rna

**E2.4**: Create a dictionary called `rc_dict` that maps DNA bases to their complementary bases. I.e., A -> T, C -> G, etc. 

In [ ]:
# Answer
rc_dict = {'A':'T', 'C':'G', 'G':'C', 'T':'A'}

# For example:
rc_dict['A']

In [ ]:
# To compute the reverse complement, we need to create a 'translation table',
# which is also a dictionary, but takes numerical ascii values as keys
# instead of strings
rc_table = str.maketrans(rc_dict)
rc_table

**E2.5**: By passing `rc_table` to the string method `.translate()`, then using indexing with a step of -1, compute the reverse complement of the MCS sequence given above.

In [ ]:
# Compute reverse complement
mcs_seq_rc = mcs_seq.translate(rc_table)[::-1]

# Print forward and RC sequences
print('FW:', mcs_seq)
print('RC:', mcs_seq_rc)

**E2.6**: We have not yet discussed sets. Using Google, figure out what `set` objects are and explain what they represent. In particular, explain why Python evaluates {2,3,3} < {1,2,3} as True.

In [ ]:
# Sets are like lists, but the elements therein don't have a specific order.
# Moreover, each element can occur at most once. 
# So {2,3,3} and {2,3} are the same set. To see this:
print('{2,3,3} == {2,3} is ', {2,3,3} == {2,3})

# The '<' sign is interpreted as Python as 'is subset'. Because
# {2,3} is a subset of {1,2,3}, this evaluates to true
print('{2,3} < {1,2,3} is ', {2,3} < {1,2,3})

## 3_flowcontrol.ipynb

Before there were calculators, people had to compute numbers like $\pi$ by hand. This was done by deriving an infinite series expression, then hand-computing the first $N$ terms using standard arithmetic. One way of computing $\pi$ was the **Leibnitz series**:

$$\pi = 4 \left(1 - \frac{1}{3} + \frac{1}{5} - \frac{1}{7} + \cdots \right) = \sum_{n=0}^\infty 4 \frac{(-1)^n}{2n+1}$$

A different way of computing $\pi$ was the **Madhava series**:

$$ \pi = \sqrt{12} \left( 1 - \frac{1}{3 \cdot 3} + \frac{1}{5 \cdot 3^2}  - \frac{1}{7 \cdot 3^3}  + \cdots \right) = \sum_{n=0}^\infty \sqrt{12} \frac{(-1)^n}{(2n+1)\cdot 3^n} $$

We will compare the accuracies of these series in two different ways.

**E3.1**: Using a `for` loop, estimate $\pi$ using the first $10$ terms in both the Liebnitz and Madhava series. Which estimate is more accurate?

In [ ]:
# Answer 

# Initialize both estimates of pi to zero
pi_L = 0
pi_M = 0

# Iterate over n, each time adding the next term to each approximation
for n in range(10):
    pi_L += 4 * (-1)**n / (2*n + 1)
    pi_M += np.sqrt(12) * (-1)**n / ((2*n+1) * 3**n)
    
# Print the results
print(f'pi_L = {pi_L:.15f}')
print(f'pi_M = {pi_M:.15f}')
print(f'pi   = {np.pi:.15f}')
print('The Madhava series is far more accurate')

**E3.2**: Write a function that computes the Leibnitz series for $\pi$ to a specified number of terms. Include a docstring and checks to make sure the input is sane.

In [ ]:
# Write function here 

def pi_leibnitz(N):
    """
    Computes the Liebnitz series for pi to N terms. 
    N must be an int >= 0.
    """
    
    # Check input
    assert isinstance(N, int), 'N must be an integer'
    assert N >= 0, 'N must be positive'
    
    # Compute terms
    ns = np.arange(N)
    terms = 4*((-1)**ns / (2*ns+1))
    approx = terms.sum()
    
    # Return estimte to user
    return approx

# To check that this is working
for N in [1, 10, 100, 1000]:
    print(f'pi_leibnitz({N}) = {pi_leibnitz(N):.10f}')

In [ ]:
# Check that the docstring works
pi_leibnitz?

In [ ]:
# Check that crazy input is caught
pi_leibnitz(.5)